<a href="https://colab.research.google.com/github/tr-dev-bc/Modern_CV_Assignments/blob/main/midterm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt         # used to plot error later
# helps running in jupyter
%matplotlib inline

from torchvision import datasets

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"         # switch for gpu else cpu

# load and normalize the fmnist dataset
data_folder = '~/FMNIST'
fmnist = datasets.FashionMNIST(data_folder, download=True, train=True)
tr_images = fmnist.data
tr_targets = fmnist.targets

In [23]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float()
        x = x.view(-1, 28 * 28) # flattens this into a vector (1D) - the view() method is used to reshape a tensor without changing its data.
                             # converts the 28x28 pixel images into a single vector of 784 elements
                             # -1 is a placeholder for one of the dimensions, allows PyTorch to automatically calculate that dimension based on the total number of elements
        self.x, self.y = x, y
    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]  # unpack tuple into variables
        return x.to(device), y.to(device)
    def __len__(self):
        return len(self.x)

In [24]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size=64, shuffle=True)         # batch size is 64
    return trn_dl
    #The DataLoader class in PyTorch loads dataset data, cuts it into batches, shuffles it

In [25]:
from torch.optim import SGD
def get_model():
  class neuralnet(nn.Module):
    def _init_(self):
      super().__init__()
      self.input_to_hidden_layer = nn.Linear(28 * 28,128) #128 neurons specified
      self.batch_norm = nn.BatchNorm1d(128)
      self.hidden_layer_activation = nn.ReLU()        #ReLU activation added
      self.hidden_to_output_layer = nn.Linear(128,10)
    def forward(self,x):
      x = self.input_to_hidden_layer(x)
      x0 = self.batch_norm(x)
      x1 = self.hidden_layer_activation(x0)
      x2 = self.hidden_to_output_layer(x1)
      return x2, x1
  model = neuralnet().to(device) # moves to the GPU to make it faster
  loss_fn = nn.CrossEntropyLoss()     # requ ired cross entropy loss function
  optimizer = SGD(model.parameters(), lr=1e-2) # model.parameters() retrieves all the parameters of the model that need to be optimized.
  return model, loss_fn, optimizer